# Concatenating, Appending, Joining DataFrames

This is a complex topic.  The examples below just demonstrate the basics -- you will need to do more investigation and/or experimentation when you have real work to do.

In [ ]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

In [ ]:
A = pd.DataFrame({"A":[1, 2, 3], "B":[4, 5, 6]})
A

In [ ]:
B = pd.DataFrame({"C":[1, 2, 3], "D":[4, 5, 6]})
B

In [ ]:
# Careful ... indices don't have to be unique -- confusing.
pd.concat([A, B], sort=False)

In [ ]:
# try extracting row '1' - using the dictionary-type (explicit index) method
pd.concat([A,B], sort=False).loc[1]

In [ ]:
# or row '1' - using Python implicit numbering style
pd.concat([A,B], sort=False).iloc[1]

In [ ]:
# Catching repeats
# We didn't go over exception handling -- look up 'try/catch' for details :-)
try:
    pd.concat([A, B], sort=False, verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# if indexes don't matter -- ignore them and create a new
# explicit index with the combined dataframe
pd.concat([A, B], sort=False, ignore_index=True)

In [ ]:
# Now the explicit index matches the implict index
pd.concat([A, B], sort=False, ignore_index=True).index

In [ ]:
# specify that the concatenation should happen along axis 1
pd.concat([A, B], axis=1)

In [ ]:
C = pd.DataFrame({"A":[7, 8, 9], "B":[10, 11, 12]})
C

In [ ]:
pd.concat([A,C])

In [ ]:
pd.concat([A,C], ignore_index=True)

In [ ]:
pd.concat([A,C], axis=1)

In [ ]:
pd.concat([A,C], axis=1)['B']

In [ ]:
pd.concat([A,C], axis=1, ignore_index=True)

In [ ]:
D = pd.DataFrame({"C":[7, 8, 9], "D":[10, 11, 12]}, index=['x', 'y', 'z'])
D

In [ ]:
E = pd.DataFrame({"A":[7, 8, 9], "D":[10, 11, 12]}, index=['x', 'y', 'z'])
E

In [ ]:
pd.concat([A,D], axis=1)

In [ ]:
pd.concat([A, E], join_axes=[A.columns])

In [ ]:
A

In [ ]:
E

## Joins

Implements a subset of <em>relational algebra</em>.  If you have a database background, you will recognize this topic.  If not, we'll do a quick run-through here and will then circle back later in the semester when we get to the MySQL component.

In [ ]:
# Create a dataframe
x = pd.DataFrame({'A' : [100, 200, 400], 'B' : [1, 3, 4]})
x

In [ ]:
# Create a second data frame with a common column ('A')
y = pd.DataFrame({'A' : [100, 200, 900, 1000], 'C' : [18, 13, 12, 22]})
y

In [ ]:
# Concatenate - try with axis = 0, 1
pd.concat([x, y], sort=False, axis=1)

In [ ]:
# With a join, we want something different -- we want to join rows that have a common
# key value so that we can use columns from both sets.
#
# Default merge - include rows where the common row values match (an 'inner join')
pd.merge(x, y)

In [ ]:
# try inner, left, right, outer joins
pd.merge(x, y, how="inner")

### Multi-table joins

In [ ]:
# Create a third data frame with a common column ('A')
z = pd.DataFrame({'A' : [100, 400, 1000, 2000], 'D' : [9,6,4,18]})
z

In [ ]:
# again, test with inner, left, right, outer -- in all combinations
pd.merge(z,pd.merge(x, y, how="inner"), how="inner")

In [ ]:
# What if I have multiple instances of a key value?
# Create a third data frame with a common column ('A')
z = pd.DataFrame({'A' : [100, 100, 100, 2000], 'D' : [9,6,4,18]})
z

In [ ]:
# Many-to-one relationship
pd.merge(z,pd.merge(x, y, how="inner"), how="inner")

## Order-Item-SKU Example

In [ ]:
# Assume that we have a dataframe with a list of orders.
orders = pd.DataFrame({
          'order' : [   123,    456,      789,    823,     950,       1024],
       'customer' : [ 'Jeff',  'Bob', 'Annie', 'Jeff', 'Chuck', 'Michelle']})
orders

In [ ]:
# And we have a separate dataframe with the items in each order
items = pd.DataFrame({
        'order' : [   123,    123,    123,    456,    456,    789,    823,     950,    950,   1024],
          'sku' : ['A109', 'A100', 'A200', 'A109', 'A227', 'A109', 'A100',  'A300', 'A904', 'A200'],
        'price' : [765.55, 227.83,  12.50, 665.55,  10.68, 760.00, 225.55, 2650.55,  15.22,  12.25]})
items

In [ ]:
# And we have a third dataframe with the item (SKU) information.
skus = pd.DataFrame({
      'sku' : [   'A100',    'A109',    'A200',    'A227',    'A300',    'A876',    'A904',   'A1021'],
    'descr' : ['Widget1', 'Widget2', 'Widget3', 'Widget4', 'Widget5', 'Widget6', 'Widget7', 'Widget8'],
     'cost' : [    12.50,    423.50,      6.50,      6.34,   1850.45,      3.23,      7.50,     18.55]})
skus

In [ ]:
# We'd like to join (merge) the data so that the item information can be 
# easily combined with the order information.
pd.merge(orders, items)

In [ ]:
# And now add in the item/SKU information (and save the resulting dataframe)
sales = pd.merge(skus, pd.merge(orders, items))
sales

In [ ]:
# now that we have the data we need, let's add a calculated column
sales['profit'] = sales['price'] - sales['cost']
sales

In [ ]:
# Grab all my stuff ...
sales[sales.customer == 'Jeff']

In [ ]:
# Total the profit ... How does this work (i.e., can you explain steps?
# What an awesome test question this would be ... :-)).
sales[sales.customer == 'Jeff'].profit.sum()

In [ ]:
sales[sales.profit > 250]

In [ ]:
# What if we use a left join on the second merge/join ...
a = pd.merge(skus, pd.merge(orders, items), how="left")
a

In [ ]:
# Now, suppose that we define a mask to identify those entries
# with no customer ...
pd.isna(a['customer'])

In [ ]:
# ... and then apply that mask to the left-join dataset ...
a[pd.isna(a['customer'])]